In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

df = pd.read_csv('./drive/MyDrive/train_labels.csv')
df.sample(3)

,Unnamed: 0,row_id,tomo_id,Motor_axis_0,Motor_axis_1,Motor_axis_2,Array_shape_axis_0,Array_shape_axis_1,Array_shape_axis_2,Voxel_spacing,Number_of_motors
513,513,405,tomo_8f4d60,120.0,450.0,457.0,300,960,928,15.6,1
257,257,681,tomo_e7c195,-1.0,-1.0,-1.0,300,960,928,16.8,0
99,99,425,tomo_97876d,-1.0,-1.0,-1.0,800,928,960,13.1,0


In [ ]:
import os

tomo_ids_list = [id for id in os.listdir('./drive/MyDrive/tomo_adaptequal_1_chunks') if id.startswith('tomo')]

In [ ]:
test_tomos = ['tomo_dae195', 'tomo_f2fa4a', 'tomo_cabaa0', 'tomo_f7f28b', 'tomo_ed1c97', 'tomo_ff505c', 'tomo_8f4d60', 'tomo_2aeb29', 'tomo_651ecd', 'tomo_e96200', 'tomo_0d4c9e', 'tomo_2dcd5c', 'tomo_983fce', 'tomo_7b1ee3', 'tomo_8b6795', 'tomo_dcb9b4', 'tomo_e764a7', 'tomo_e26c6b', 'tomo_331130', 'tomo_f8b835', 'tomo_746d88', 'tomo_9cd09e', 'tomo_b9eb9a', 'tomo_cf0875', 'tomo_7cf523', 'tomo_fd41c4', 'tomo_54e1a7', 'tomo_ca472a', 'tomo_6478e5', 'tomo_e9b7f2', 'tomo_247826', 'tomo_675583', 'tomo_f0adfc', 'tomo_378f43', 'tomo_19a313', 'tomo_172f08', 'tomo_f3e449', 'tomo_3b83c7', 'tomo_8c13d9', 'tomo_2c607f', 'tomo_c11e12', 'tomo_412d88', 'tomo_4b124b', 'tomo_38c2a6', 'tomo_ec1314', 'tomo_1c38fd', 'tomo_e63ab4', 'tomo_f07244', 'tomo_210371', 'tomo_d6e3c7', 'tomo_935f8a', 'tomo_a4c52f', 'tomo_a46b26', 'tomo_fadbe2', 'tomo_b28579', 'tomo_35ec84', 'tomo_369cce', 'tomo_6c203d', 'tomo_b80310', 'tomo_640a74', 'tomo_22976c', 'tomo_d21396', 'tomo_ecbc12', 'tomo_040b80', 'tomo_85708b', 'tomo_b98cf6', 'tomo_e1e5d3', 'tomo_138018', 'tomo_3264bc', 'tomo_e50f04', 'tomo_d723cd', 'tomo_2a6ca2', 'tomo_1f0e78', 'tomo_67565e', 'tomo_fd5b38', 'tomo_05b39c', 'tomo_372a5c', 'tomo_c3619a', 'tomo_ba76d8', 'tomo_a67e9f', 'tomo_a6646f', 'tomo_db656f', 'tomo_4102f1', 'tomo_bb5ac1', 'tomo_4ed9de', 'tomo_61e947', 'tomo_1da0da', 'tomo_821255', 'tomo_3e7783', 'tomo_c84b46', 'tomo_974fd4', 'tomo_444829', 'tomo_b50c0f', 'tomo_2a6091', 'tomo_fa5d78', 'tomo_bdd3a0', 'tomo_1c2534', 'tomo_d916dc', 'tomo_bdc097', 'tomo_7036ee', 'tomo_cacb75', 'tomo_5b359d', 'tomo_7fa3b1', 'tomo_049310', 'tomo_dd36c9', 'tomo_e3864f', 'tomo_0a8f05', 'tomo_ff7c20', 'tomo_0fab19', 'tomo_1c75ac', 'tomo_d0699e', 'tomo_1e9980', 'tomo_4ee35e', 'tomo_6943e6', 'tomo_99a3ce']

In [ ]:
tomos_train_val = [tomo_id for tomo_id in tomo_ids_list if tomo_id not in test_tomos]

In [ ]:
val_cut = round(len(tomos_train_val)/5)

In [ ]:
val_tomos = tomos_train_val[:val_cut]
train_tomos = tomos_train_val[val_cut:]

In [ ]:
len(test_tomos), len(val_tomos), len(train_tomos)

(115, 69, 277)

In [ ]:
lists = [test_tomos, val_tomos, train_tomos]

for idx, list_ in enumerate(lists):
    filtered = []
    for id in list_:
        row = df[df['tomo_id'] == id]
        if not row.empty and row.iloc[0]['Number_of_motors'] != 0:
            filtered.append(id)
    lists[idx] = filtered

test_tomos_filtered = lists[0]
val_tomos_filtered = lists[1]
train_tomos_filtered = lists[2]

In [ ]:
len(test_tomos_filtered), len(val_tomos_filtered), len(train_tomos_filtered)

(60, 49, 210)

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.utils import load_img, img_to_array

def load_tomo_sequence(tomo_path, img_size=(128, 128)):
    slices = sorted(os.listdir(tomo_path))
    images = []
    for s in slices:
        img = load_img(os.path.join(tomo_path, s), color_mode='grayscale', target_size=img_size)
        arr = img_to_array(img) / 255.0
        images.append(arr)
    return np.stack(images)  # shape: (n_slices, H, W, C)

def build_X_y_image_sequence(tomo_ids, base_dir, df_labels, img_size=(128, 128)):
    X = []
    y = []
    for tomo_id in tomo_ids:
        path = os.path.join(base_dir, tomo_id)
        X.append(load_tomo_sequence(path, img_size))
        y.append(df_labels[df_labels["tomo_id"] == tomo_id]["Motor_axis_0"].values[0])
    return np.array(X), np.array(y)


In [ ]:
base_dir = "./drive/MyDrive/tomo_adaptequal_1_chunks"

img_size = (960, 960)  # ou (960, 960) selon ton modèle

X_train, y_train = build_X_y_image_sequence(train_tomos_filtered, base_dir, df, img_size)
X_val, y_val = build_X_y_image_sequence(val_tomos_filtered, base_dir, df, img_size)
X_test, y_test = build_X_y_image_sequence(test_tomos_filtered, base_dir, df, img_size)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, TimeDistributed, Conv2D, MaxPooling2D, Flatten, LSTM, Dense

timesteps = X_train.shape[1]
height = X_train.shape[2]
width = X_train.shape[3]
channels = X_train.shape[4]

inputs = Input(shape=(timesteps, height, width, channels))

# CNN appliqué à chaque image de la séquence
x = TimeDistributed(Conv2D(16, (3,3), activation='relu'))(inputs)
x = TimeDistributed(MaxPooling2D((2,2)))(x)
x = TimeDistributed(Conv2D(32, (3,3), activation='relu'))(x)
x = TimeDistributed(MaxPooling2D((2,2)))(x)
x = TimeDistributed(Flatten())(x)

# LSTM sur les représentations d'images
x = LSTM(64)(x)
x = Dense(32, activation='relu')(x)
outputs = Dense(1)(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=4)
